In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time
import matplotlib.pyplot as plt
#from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
#from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16;
from keras.applications.vgg16 import preprocess_input
import os

In [5]:
mnist = tf.keras.datasets.mnist

In [8]:
(trainX, trainY), (testX, testY) = mnist.load_data()
print(trainX.shape, testX.shape)

(60000, 28, 28) (10000, 28, 28)


In [10]:
trainX=np.dstack([trainX] * 3)
testX=np.dstack([testX]*3)
trainX.shape,testX.shape

((60000, 28, 84), (10000, 28, 84))

In [11]:
# Reshape images as per the tensor format required by tensorflow
trainX = trainX.reshape(-1, 28,28,3)
testX= testX.reshape (-1,28,28,3)
trainX.shape,testX.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [12]:
# Resize the images 48*48 as required by VGG16
from keras.preprocessing.image import img_to_array, array_to_img
trainX = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in trainX])
testX = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in testX])
#train_x = preprocess_input(x)
trainX.shape, testX.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

In [14]:
# Normalise the data and change data type
trainX = trainX / 255.
testX = testX / 255.
trainX = trainX.astype('float32')
testX = testX.astype('float32')

In [15]:
# Converting Labels to one hot encoded format
trainY_one_hot = to_categorical(trainY)
testY_one_hot = to_categorical(testY)

trainX,valid_X,train_label,valid_label = train_test_split(trainX,
                                                           trainY_one_hot,
                                                           test_size=0.2,
                                                           random_state=13
                                                           )

In [23]:
# Finally check the data size whether it is as per tensorflow and VGG16 requirement
trainX.shape,valid_X.shape,train_label.shape,valid_label.shape

# Define the parameters for instanitaing VGG16 model. 
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

# Preprocessing the input 
trainX = preprocess_input(trainX)
valid_X = preprocess_input(valid_X)
testX  = preprocess_input (testX)

#  Create base model of VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False, 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)
                 )
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     